In [2]:
import os

# Replace with the actual path to your JSON key file
credentials_path = "/content/caixa-bank-6c4f815e89ac.json"

# Check if the file exists
if not os.path.exists(credentials_path):
    raise FileNotFoundError(f"Credentials file not found: {credentials_path}")

# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

# **Important**: Restart the kernel or runtime environment after setting the environment variable.
# This ensures that the changes are picked up by the BigQuery client library.

FileNotFoundError: Credentials file not found: /content/caixa-bank-6c4f815e89ac.json

In [3]:
from google.cloud import  bigquery
import pandas as pd

In [4]:
client = bigquery.Client()

In [5]:
query = """ SELECT * FROM `caixa-bank.ML.risk_table`"""

In [6]:
risk_df = client.query(query).to_dataframe()

RefreshError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7916a1af2560>)

In [ ]:
risk_df.head()

In [ ]:
risk_df['debt_to_income_ratio'] = risk_df['total_debt'] / risk_df['yearly_income'] # Customers with higher debt-to-income ratios are generally riskier.
risk_df['credit_utilization'] = risk_df['total_debt'] / (risk_df['num_credit_cards'] * risk_df['credit_score'])# Customers with higher utilization (close to 1 or more) are generally riskier.
risk_df['age_to_retirement_ratio'] = (risk_df['retirement_age'] - risk_df['current_age']) / risk_df['retirement_age']#Younger customers may behave differently than those nearing retirement.
risk_df['has_fraud_history'] = risk_df['fraud_count'].apply(lambda x: 1 if x > 0 else 0)
risk_df['debt_credit_interaction'] = risk_df['total_debt'] * risk_df['credit_score']




In [ ]:
risk_df.head()


In [ ]:
gender_encoded = pd.get_dummies(risk_df['gender'], prefix='gender')


risk_df = pd.concat([risk_df, gender_encoded], axis=1)


risk_df = risk_df.drop(columns=['gender'])




In [ ]:
risk_df.head()


In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target
X = risk_df[['current_age', 'yearly_income', 'total_debt', 'credit_score', 'per_capita_income',
             'debt_to_income_ratio', 'age_to_retirement_ratio', 'has_fraud_history', 'gender_Male', 'gender_Female']]
y = risk_df['risk_category'].map({'Low Risk': 0, 'Medium Risk': 1, 'High Risk': 2})

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model
model.fit(X_train_scaled, y_train)


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test_scaled)


In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
